# Tensorboard
`pip install tensorboardX`  
`pip install tensorflow`  
`pip install Tensorboard_logger`  

In [1]:
import torch
from torchvision.datasets import mnist
from torch.utils.data import DataLoader
import numpy as np
import torch.nn.functional as F
from torch import nn
import matplotlib.pyplot as plt
from tensorboardX import SummaryWriter 
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))
def data_transform(x):
    x = np.array(x, dtype = 'float32') / 255
    x = x.reshape((-1, ))
    x = torch.from_numpy(x)
    return x
trainset = mnist.MNIST('./dataset/mnist', train=True, transform=data_transform, download=True)
testset = mnist.MNIST('./dataset/mnist', train = False, transform=data_transform, download=True)
train_data = DataLoader(trainset, batch_size=64, shuffle=True)
test_data = DataLoader(testset, batch_size=128, shuffle=False)
# tensorboard ======================
writer = SummaryWriter('./tensorboard_logs2')

class DNN(nn.Module):
    def __init__(self):
        super(DNN, self).__init__()
        self.fc1 = nn.Linear(28*28, 500)
        self.fc2 = nn.Linear(500, 250)
        self.fc3 = nn.Linear(250, 125)
        self.fc4 = nn.Linear(125, 10)
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x
# build model
model = DNN().to(device)
# define cost
criterion = nn.CrossEntropyLoss()
# optimization
optimizer = torch.optim.SGD(model.parameters(), 0.1)
#optimizer = torch.optim.RMSprop(model.parameters(), 0.01, alpha=0.9)
#optimizer = torch.optim.Adam(model.parameters(), 0.01, betas=(0.9, 0.99))
losses = []
acces = []
eval_losses = []
eval_acces = []
for epoch in range(5):
    train_loss = 0
    train_acc = 0
    model.train()      
    for index, (im, label) in enumerate(train_data):
        im = im.to(device)
        label = label.to(device)
        pred = model(im)
        loss = criterion(pred, label)
        # tensorboard ======================
        writer.add_scalar('Loss', loss, epoch * len(trainset.data)//64 + index)
        # zero the parameter gradients
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        _, pred = pred.max(1)
        num_correct = (pred == label).sum().item()
        acc = num_correct / im.shape[0]
        # tensorboard ======================
        writer.add_scalar('Accuracy', acc,  epoch * len(trainset.data)//64 + index)
        train_acc += acc
    losses.append(train_loss / len(train_data))
    acces.append(train_acc / len(train_data))
    eval_loss = 0
    eval_acc = 0
    model.eval()
    for im, label in test_data:
        im = im.to(device)
        label = label.to(device)
        pred = model(im)
        loss = criterion(pred, label)
        eval_loss += loss.item()
        _, pred = pred.max(1)
        num_correct = (pred == label).sum().item()
        acc = num_correct / im.shape[0]
        eval_acc += acc
    
    eval_losses.append(eval_loss / len(test_data))
    eval_acces.append(eval_acc / len(test_data))
    print('epoch: {}, Train Loss: {:.6f}, Train Acc: {:.6f}, Eval Loss: {:.6f}, Eval Acc: {:.6f}'
          .format(epoch, train_loss / len(train_data), train_acc / len(train_data), eval_loss / len(test_data), eval_acc / len(test_data)))

Using cpu device
epoch: 0, Train Loss: 0.586497, Train Acc: 0.826609, Eval Loss: 0.209736, Eval Acc: 0.935918
epoch: 1, Train Loss: 0.155195, Train Acc: 0.953591, Eval Loss: 0.239490, Eval Acc: 0.925237
epoch: 2, Train Loss: 0.101665, Train Acc: 0.969916, Eval Loss: 0.110857, Eval Acc: 0.964003
epoch: 3, Train Loss: 0.074364, Train Acc: 0.977195, Eval Loss: 0.084747, Eval Acc: 0.973794
epoch: 4, Train Loss: 0.057533, Train Acc: 0.982493, Eval Loss: 0.077120, Eval Acc: 0.976167
